# Bookworm Notebook

## Import Statements

In [33]:
import pandas as pd
import json

## Defining Constants

In [34]:
CONFIG_PATH = "config.json"
ORDERED_COL_NAMES = ["Account", "Date", "Category", "Tags", "Cost", "Description"]

MONTH = "august"

WF_CHECKING_CSV_PATH = "data/" + MONTH + "/input/wf-checking.csv"
WF_SAVINGS_CSV_PATH = "data/" + MONTH + "/input/wf-savings.csv"

AMEX_GOLD_CSV_PATH = "data/" + MONTH + "/input/amex-gold.csv"
AMEX_BLUE_CSV_PATH = "data/" + MONTH + "/input/amex-blue.csv"

PREPROCESSED_TRANSACTIONS_CSV_PATH = "data/" + MONTH + "/output/notebook-preprocessed-transactions.csv"
PROCESSED_TRANSACTIONS_CSV_PATH = "data/" + MONTH + "/transactions.csv"

SUMMARY_TXT_FILE_PATH = "data/" + MONTH + "/output/notebook-summary.txt"

## Reading Configuration

In [35]:
# reading configuration from a json file
with open(CONFIG_PATH, "r") as file:
    config = json.load(file)

CATEGORIES = config["Categories"]
TAGS = config["Tags"]
PRECENDENCES = config["Precendences"]

# Preprocess

## Normalizing Data

In [36]:
def normalize_wells_fargo_data(account_name, csv_file_path):
    WF_COL_INDEXES = [0, 1, 4]
    WF_COL_NAMES = ["Date", "Cost", "Description"]

    wf_data = pd.read_csv(
        csv_file_path,
        header = None,
        usecols = WF_COL_INDEXES,
        names = WF_COL_NAMES
    )

    wf_data["Account"] = account_name
    wf_data["Category"] = ""
    wf_data["Tags"] = ""
    wf_data["Cost"] = wf_data["Cost"] * -1

    return wf_data[ORDERED_COL_NAMES]

In [37]:
def normalize_amex_data(account_name, csv_file_path):
    amex_data = pd.read_csv(csv_file_path)

    amex_data.rename(
        columns = {"Amount": "Cost"},
        inplace = True
    )

    amex_data["Account"] = account_name
    amex_data["Category"] = ""
    amex_data["Tags"] = ""

    return amex_data[ORDERED_COL_NAMES]

In [38]:
def normalize_chase_data():
    CHASE_CSV_PATH = "data/" + MONTH + "/input/chase.csv"
    CHASE_ACCOUNT_NAME = "Chase"

    CHASE_COL_INDEXES = [0, 2, 5]
    CHASE_COL_NAMES = ["Date", "Description", "Cost"]

    chase_data = pd.read_csv(
        CHASE_CSV_PATH,
        usecols = CHASE_COL_INDEXES
    )

    chase_data.columns = CHASE_COL_NAMES

    chase_data["Account"] = CHASE_ACCOUNT_NAME
    chase_data["Category"] = ""
    chase_data["Tags"] = ""
    chase_data["Cost"] = chase_data["Cost"] * -1

    return chase_data[ORDERED_COL_NAMES]

In [39]:
def normalize_apple_data():
    APPLE_CSV_PATH =  "data/" + MONTH + "/input/apple.csv"
    APPLE_ACCOUNT_NAME = "Apple"

    APPLE_COL_INDEXES = [0, 2, 6]
    APPLE_COL_NAMES = ["Date", "Description", "Cost"]

    apple_data = pd.read_csv(
        APPLE_CSV_PATH,
        usecols = APPLE_COL_INDEXES
    )

    apple_data.columns = APPLE_COL_NAMES

    apple_data["Account"] = APPLE_ACCOUNT_NAME
    apple_data["Category"] = ""
    apple_data["Tags"] = ""

    return apple_data[ORDERED_COL_NAMES]

In [40]:
def normalize_discover_data():
    DISCOVER_CSV_PATH  = "data/" + MONTH + "/input/discover.csv"
    DISCOVER_ACCOUNT_NAME = "Discover"

    DISCOVER_COL_INDEXES = [0, 2, 3]
    DISCOVER_COL_NAMES = ["Date", "Description", "Cost"]

    discover_data = pd.read_csv(
        DISCOVER_CSV_PATH,
        usecols = DISCOVER_COL_INDEXES
    )

    discover_data.columns = DISCOVER_COL_NAMES

    discover_data["Account"] = DISCOVER_ACCOUNT_NAME
    discover_data["Category"] = ""
    discover_data["Tags"] = ""

    return discover_data[ORDERED_COL_NAMES]

In [41]:
def normalize_data():
    checking = normalize_wells_fargo_data("Wells Fargo Checking", WF_CHECKING_CSV_PATH)
    savings = normalize_wells_fargo_data("Wells Fargo Savings", WF_SAVINGS_CSV_PATH)

    amex_gold = normalize_amex_data("Amex Gold", AMEX_GOLD_CSV_PATH)
    amex_blue = normalize_amex_data("Amex Blue", AMEX_BLUE_CSV_PATH)

    chase = normalize_chase_data()
    apple = normalize_apple_data()
    discover = normalize_discover_data()

    return pd.concat([checking, savings, amex_gold, amex_blue, chase, apple, discover], ignore_index=True)

## Tagging Transactions

In [42]:
def assign_tags(row, col, categories):
    tags = []

    for category, keywords in categories.items():
        if any(keyword in row[col] for keyword in keywords):
            tags.append(category)

    return ";".join(tags)

## Main Logic

In [43]:
def preprocess_transactions():
    # normalizing data
    preprocessed_transactions = normalize_data()

    # convert all strings to uppercase
    preprocessed_transactions = preprocessed_transactions.applymap(lambda value: value.upper() if isinstance(value, str) else value)

    # assigning tags
    preprocessed_transactions["Tags"] = preprocessed_transactions.apply(lambda row: assign_tags(row, "Description", TAGS), axis=1)
    preprocessed_transactions["Category"] = preprocessed_transactions.apply(lambda row: assign_tags(row, "Tags", CATEGORIES), axis=1)

    # replacing empty categories w/ misc
    preprocessed_transactions["Category"] = preprocessed_transactions["Category"].replace("", "Misc")

    # saving preprocessed data
    preprocessed_transactions.to_csv(PREPROCESSED_TRANSACTIONS_CSV_PATH, index=False)

preprocess_transactions()

# Summarize

## Creating Tag Costs Data

In [44]:
def reduce_tags(tags):
    if pd.isna(tags) or tags == "":
        return ""
    
    tag_list = tags.split(";")
    
    tag_precedence = { tag: PRECENDENCES.get(tag, float("inf")) for tag in tag_list }

    selected_tag = max(tag_precedence, key = tag_precedence.get)

    return selected_tag

In [45]:
def create_tag_costs(transactions):
    transactions["Tags"] = transactions["Tags"].apply(reduce_tags)

    tag_costs = transactions.groupby(["Category", "Tags"])["Cost"].sum().reset_index()
    tag_costs.columns = ["Category", "Tag", "Total Cost"]

    return tag_costs.sort_values(by = "Total Cost", ascending = False)

## Formatting Output

In [46]:
SEPARATOR = "-" * 33

def format_currency(num):
    num_rounded = round(num, 2)
    return f"{num_rounded:.2f}"

def create_tags_summary(tag_costs, category):
    summary_text = ""

    for _, row in tag_costs[tag_costs["Category"] == category].iterrows():
        tag = row["Tag"]
        cost = row["Total Cost"]

        summary_text += f"{tag:<20} : {format_currency(cost):>10}\n"

    return summary_text

def create_needs_summary(total_needs, needs_budget_amount, tag_costs):
    needs_difference = needs_budget_amount - total_needs

    summary_text = f"\n{'NEEDS':<20} : {format_currency(total_needs):>10}"
    summary_text += f"\n{'EXPECTED NEEDS':<20} : {format_currency(needs_budget_amount):>10}\n"

    needs_over_under = "under" if needs_difference >= 0 else "over"
    summary_text += f"\nAmount {needs_over_under} by {format_currency(needs_difference)}\n\n"

    summary_text += create_tags_summary(tag_costs, "Needs")
    
    summary_text += f"\n{SEPARATOR}\n"

    return summary_text

def create_wants_summary(total_wants, wants_budget_amount, tag_costs):
    wants_difference = wants_budget_amount - total_wants

    summary_text = f"\n{'WANTS':<20} : {format_currency(total_wants):>10}"
    summary_text += f"\n{'EXPECTED WANTS':<20} : {format_currency(wants_budget_amount):>10}\n"
    
    wants_over_under = "under" if wants_difference >= 0 else "over"
    summary_text += f"\nAmount {wants_over_under} by {format_currency(wants_difference)}\n\n"

    summary_text += create_tags_summary(tag_costs, "Wants")

    summary_text += f"\n{SEPARATOR}\n"

    return summary_text

def create_savings_summary(total_savings, savings_budget_amount, tag_costs):
    savings_difference = savings_budget_amount - total_savings

    summary_text = f"\n{'SAVINGS':<20} : {format_currency(total_savings):>10}"
    summary_text += f"\n{'EXPECTED SAVINGS':<20} : {format_currency(savings_budget_amount):>10}\n"
    
    savings_over_under = "under" if savings_difference >= 0 else "over"
    summary_text += f"\nAmount {savings_over_under} by {format_currency(savings_difference)}\n\n"

    summary_text += create_tags_summary(tag_costs, "Savings")
    summary_text += f"\n{SEPARATOR}\n"

    return summary_text

def create_summary(transactions):
    # getting costs by category
    total_costs = transactions.groupby("Category")["Cost"].sum().reset_index()
    total_costs.columns = ["Category", "Total Cost"]

    # getting the total income made for this month
    total_income = total_costs[total_costs["Category"] == "Income"]["Total Cost"].sum() * -1
    # creating summary text
    summary_text = f"\n{MONTH.upper()} EXPENSES OVERVIEW\n\n{SEPARATOR}\n"
    # adding income to summary
    summary_text += f"INCOME: {format_currency(total_income)}\n{SEPARATOR}\n"

    # getting costs by tag
    tag_costs = create_tag_costs(transactions[transactions["Category"] != "Income"].copy())

    # creating summary for needs
    total_needs = total_costs[total_costs["Category"] == "Needs"]["Total Cost"].sum()
    needs_budget_amount = total_income * .5
    summary_text += create_needs_summary(total_needs, needs_budget_amount, tag_costs)

    # creating summary for wants
    total_wants = total_costs[total_costs["Category"] == "Wants"]["Total Cost"].sum()
    wants_budget_amount = total_income * .3
    summary_text += create_wants_summary(total_wants, wants_budget_amount, tag_costs)
    
    # creating summary for savings
    total_savings = total_costs[total_costs["Category"] == "Savings"]["Total Cost"].sum()
    savings_budget_amount = total_income * .2
    summary_text += create_savings_summary(total_savings, savings_budget_amount, tag_costs)

    return summary_text

## Main Logic

In [47]:
def generate_expenses_summary():
    # manually processed data:
    #   updated categories + added tags
    #   removed rows creating 0 sums like transfers
    #   added rows for savings
    transactions = pd.read_csv(PROCESSED_TRANSACTIONS_CSV_PATH)

    # replacing empty tags w/ misc
    transactions["Tags"] = transactions["Tags"].replace("", "Misc")

    # create summarized data
    summary_txt = create_summary(transactions)

    # saving summarized data
    with open(SUMMARY_TXT_FILE_PATH, "w") as file:
        file.write(summary_txt)
    
generate_expenses_summary()

## Explore Transaction Data

In [48]:
transactions = pd.read_csv(PROCESSED_TRANSACTIONS_CSV_PATH)

transactions[transactions["Description"].str.contains("XBOX")]["Cost"].sum()

150.09